# Groupby 4: The `LazyGroupBy` object
By the end of this lecture you will be able to:
- do `group_by` in lazy mode
- do aggregations on a `LazyGroupBy`
- inspect the optimized query plan
- profile a query


In [ ]:
import polars as pl

In [ ]:
csv_file = "../data/titanic.csv"

## Creating a `LazyGroupBy` object

We create a `LazyGroupBy` object by calling `group_by` on a `LazyFrame`

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
)

## Aggregations
The only way to do aggregations on a `LazyGroupBy` is with `agg`. We cannot call, for example, `.mean` on `.groubpy` as we can with an eager group_by. 

Calling `agg` converts a `LazyGroupBy` to a `LazyFrame`

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
)

### Query optimizations
We print the optimized plan for this group_by query

In [ ]:
print(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
    .explain()
)

In the optimized plan we have:
- `PROJECT 2/12 COLUMNS` so Polars will only read the `Pclass` and `Age` columns from the CSV
- `Aggregate [col("Age").mean()] BY [col("Pclass")]` so Polars will group by the `Pclass` column and take the `mean` of the `Age` column

As with any lazy query we can evaluate this either all-at-once or in batches using streaming. To evaluate all-at-once call `collect` and to evaluate with streaming call `collect(streaming=True)`. As noted in the first lecture on group_by, streaming will not work if we set `maintain_order=True`

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
    .collect(streaming=True)
)

## Profiling a lazy query
For any lazy query Polars can profile the query showing how long each part of the query takes.

We demonstrate this in the context of a lazy group_by.

The output of `profile` is a 2-element tuple. The first element of the tuple is the output of the query - the same output as we get from `collect` 

In [ ]:
grouped_df,profile_df = (
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
    .profile()
)
grouped_df

The second element is a `DataFrame` with timings in microseconds for the start and end of each node in the optimised query plan

In [ ]:
profile_df

Polars can also generate a Gantt chart for the timings. For this you need to have Matplotlib installed. The output in this case is a tuple of the chart object and the timings `DataFrame`. We assign these to separate variables here

In [ ]:
chart,timings_df =(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
    .sort("Pclass")
    .profile(show_plot=True,figsize=(6,3))
)
chart

We inspect the timings `DataFrame` after adding on a column to measure how long each segment takes

In [ ]:
pl.Config.set_fmt_str_lengths(100)
(
    timings_df
    .with_columns(segment_time=pl.col("end")-pl.col("start"))
)

In the chart we see that the time required for:
- optimization of the query is relatively small
- reading the CSV is the largest component
- doing the group_by and sort take a similar amount of time

We can also profile the code when it is run in streaming by setting `streaming=True` in `profile`

In [ ]:
chart_streaming,timings_streaming_df =(
    pl.scan_csv(csv_file)
    .group_by('Pclass')
    .agg(
        pl.col("Age").mean()
    )
    .sort("Pclass")
    .profile(show_plot=True,figsize=(6,3),streaming=True)
)
chart_streaming

In streaming mode we get the output of the part of the code run in streaming mode as a single node called `STREAMING`

In [ ]:
timings_streaming_df

## Exercises
In the exercises you will develop your understanding of:
- creating a `LazyGroupBy`
- doing an aggregation on a `LazyGroupBy`
- interpreting optimized query plans
- profiling a query

## Exercise 1
Create a `LazyGroupBy` on the `Survived` and `Plcass` columns in a query that starts with scanning the CSV

Aggregate the data by getting the minimum, average and maximum age per group

Evaluate the query

Evaluate the query and produce a profile plot

## Exercise 2
Create the query that has the following optimized plan:

```
SORT BY [col("Decade")]
  AGGREGATE
  	[col("Fare").mean()] BY [[(col("Age")) / (10.0)].round().strict_cast(Int64).alias("Decade")] FROM
    DROP_NULLS by: [Age]

        Csv SCAN ../data/titanic.csv
        PROJECT 2/12 COLUMNS
```

In [ ]:
print(
    <blank>
    .explain()
)

## Solutions

## Solution to Exercise 1
Create a `LazyGroupBy` on the `Survived` and `Plcass` columns in a query that starts with scanning the CSV

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
)

Aggregate the data by getting the minimum, average and maximum age per group

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
    .agg(
        [
            pl.col("Age").min().name.suffix("_min"),
            pl.col("Age").mean().name.suffix("_mean"),
            pl.col("Age").max().name.suffix("_max"),
        ]
    )
)

Evaluate the query

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
    .agg(
        [
            pl.col("Age").min().name.suffix("_min"),
            pl.col("Age").mean().name.suffix("_mean"),
            pl.col("Age").max().name.suffix("_max"),
        ]
    )
    .collect()
)

Evaluate the query and produce a profile plot

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
    .agg(
        [
            pl.col("Age").min().name.suffix("_min"),
            pl.col("Age").mean().name.suffix("_mean"),
            pl.col("Age").max().name.suffix("_max"),
        ]
    )
    .profile(show_plot=True)
)

## Solution to Exercise 2
Create the query with the optimized plan

```
SORT BY [col("Decade")]
  AGGREGATE
  	[col("Fare").mean()] BY [[(col("Age")) / (10.0)].round().strict_cast(Int64).alias("Decade")] FROM
    DROP_NULLS by: [Age]

        Csv SCAN ../data/titanic.csv
        PROJECT 2/12 COLUMNS
```

In [ ]:
print(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age").is_not_null())
    .group_by(
        (pl.col("Age")/10).round(0).cast(pl.Int64).alias("Decade")
    )
    .agg(
        pl.col("Fare").mean()
    )
    .sort("Decade",descending=True)
    # .collect()
    .explain()
)